Memory Module

In [1]:
## Import functions
import os
import openpyxl
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from openai import OpenAI
import tiktoken
import json
from llm_demos.llama2.embeddings import llama_embed
from llm_demos.llama2.gguf import importance_f

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/frattitamayo/.cache/huggingface/hub/models--NousResearch--Hermes-2-Pro-Mistral-7B-GGUF/snapshots/594e3e33f57a2b8693972e6bf48ae4eff404f170/Hermes-2-Pro-Mistral-7B.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = jeffq
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:

In [2]:
def get_llama_embedding(document:str):
    return llama_embed([document])

In [3]:
## Get data from files

# Replace 'your_file_path.xlsm' with the path to your Excel file
file_path = '/home/frattitamayo/memory_module/CodingDiscourseAnalysis/CollabWriteAnalysisTest.xlsm'

# Load the Excel file
df = pd.read_excel(file_path, sheet_name='Test', parse_dates=['Message Time'])
observations = []
times = []

for index, row in df.iterrows():
    # Construct the observation for each row
    time_origin = row['Message Time']
    origin = f"Question: {row['Topic']}, Pseudonym: {row['Pseudonym']}"
    # origin = f"{row['Pseudonym']}"
    value = row['Message']
    observation = (time_origin, origin, value)
     #observation = [(origin_1, value_1), (origin_2, value_2), ..., (origin_n, value_n)]
    observations.append(observation)
    
print(observations)

[(Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred', 'Hello. '), (Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred', 'My assumption is'), (Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred ', 'that the emphasis on barbarism implies that she sent him to the lion.'), (Timestamp('2020-10-27 17:58:00'), 'Question: Part 1: What happens next? What is behind

Get observations

In [4]:
from itertools import zip_longest
GAMMA = 0.999 # Decay factor between [0-1]

class Observation:
    def __init__(self, time_origin, origin, value, ): # Origin, Emitter, Value
        self.time_origin = time_origin
        self.recency_value = 1
        self.origin = origin # Where it came from
        self.value = value # What it contains
        self.embed_vector = None
        self.get_embed_vector()

    def __str__(self,): #{self.origin}
        return "{"+ f"[{self.time_origin}] {self.origin}: \"{self.value}\"" + "}"
    
    
    def get_embed_vector(self,):
        if self.embed_vector is None:
            print(self.__str__(), "to embedding...")
            self.embed_vector = get_llama_embedding(self.__str__())
        return self.embed_vector
        
    def decay_recency(self, gamma):
        self.recency_value *= gamma

observations1 = [Observation(idx, origin, value) for idx, (time_origin, origin, value) in enumerate(observations)]
#observations1 = [Observation(idx+1, origin, value) for idx, (origin, value) in enumerate(observations)]

# for i in observations1:
#     print(i.__str__())


{[0] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "Hello. "} to embedding...



llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1905.30 ms /    73 tokens (   26.10 ms per token,    38.31 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1905.84 ms /    74 tokens


{[1] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "My assumption is"} to embedding...



llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1763.09 ms /    73 tokens (   24.15 ms per token,    41.40 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1763.32 ms /    74 tokens


{[2] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred : "that the emphasis on barbarism implies that she sent him to the lion."} to embedding...


...


llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1917.79 ms /    79 tokens (   24.28 ms per token,    41.19 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1917.98 ms /    80 tokens


{[7] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Marissa Roswell: " Sadly  the barbaric nature of the king and princess is still alive today in crimes of passion.""} to embedding...



llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    2208.66 ms /    91 tokens (   24.27 ms per token,    41.20 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2208.76 ms /    92 tokens


{[8] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: "Submitted"} to embedding...



llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1758.94 ms /    72 tokens (   24.43 ms per token,    40.93 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1758.67 ms /    73 tokens


In [5]:
## Helper functions

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [6]:
def query_k_obs(query: str, k: int, curr: int):
    del observations1[curr]
    query_embedding = get_llama_embedding(query)
    relevance = [cosine_similarity(query_embedding, obs.get_embed_vector().T) for obs in observations1]
    importance = [importance_f(query, obs.__str__()) for obs in observations1]
    recency = [obs.recency_value for obs in observations1]

    print(relevance)
    for idx, (rel, imp, rec) in enumerate(zip(relevance, importance, recency)):
        print((idx, rel+imp+rec))
  

    values = sorted(([(idx, rel+imp+rec) for idx, (rel, imp, rec) in enumerate(zip(relevance, importance, recency))]), key=lambda x: x[1], reverse=True)
    print(values)

    #rel+imp+rec replace in function 
    k_best_obs = [observations1[v[0]] for v in values[:k]] 

    for obs in observations1:
        obs.decay_recency(GAMMA)

    for obs in k_best_obs:
        obs.recency_value = 1

    return k_best_obs

In [7]:

def memory_module(observations):
    for indx, (obs) in enumerate(observations):
        k_best_obs = query_k_obs(observations1[indx].__str__(), 3, indx)
        for i in k_best_obs:
            print(i.__str__())
        break
    


print(memory_module(observations1))


llama_print_timings:        load time =     193.83 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1773.80 ms /    73 tokens (   24.30 ms per token,    41.15 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1773.94 ms /    74 tokens


Rationale: These two messages were sent by the same person but they are unrelated. The second message doesn't provide any additional information related to the query since it's just a greeting.


llama_print_timings:        load time =     219.93 ms
llama_print_timings:      sample time =       5.21 ms /    42 runs   (    0.12 ms per token,  8053.69 tokens per second)
llama_print_timings: prompt eval time =   24158.33 ms /   839 tokens (   28.79 ms per token,    34.73 tokens per second)
llama_print_timings:        eval time =    5827.30 ms /    41 runs   (  142.13 ms per token,     7.04 tokens per second)
llama_print_timings:       total time =   30207.09 ms /   880 tokens


I

Llama.generate: prefix-match hit


...

Note: Please only respond in the format "I: (importance value)" if you want to get points. If you do any other way, you will not get any points.
I: 4.5
Rationale: Although both messages contain the same question and pseudonym, they don't continue from each other but instead contain completely different statements that don't necessarily complement each other. Therefore, they are moderately important to know together but not extremely important. This results in a score of 4.5.


llama_print_timings:        load time =     219.93 ms
llama_print_timings:      sample time =      13.53 ms /   109 runs   (    0.12 ms per token,  8058.55 tokens per second)
llama_print_timings: prompt eval time =    4873.52 ms /   168 tokens (   29.01 ms per token,    34.47 tokens per second)
llama_print_timings:        eval time =   15354.68 ms /   108 runs   (  142.17 ms per token,     7.03 tokens per second)
llama_print_timings:       total time =   20516.43 ms /   276 tokens
Llama.generate: prefix-match hit


Rationale: In this case, both observations and queries are the same except for the pseudonyms and the words 'Submitted' and 'Hello'. However, these words don't significantly change the meaning or importance of the query given the prior. Thus, knowing the prior (the query) isn't much more important than not knowing it, and so the importance should be low.
I: 2
Rationale: In this case, both observations and queries are nearly identical except for the pseudonyms and the words 'Submitted' and 'Hello'. However, these words don't significantly change the meaning or importance of the query given the prior. Thus, knowing the prior (the query) isn't much more important than not knowing it, and so the importance should be low.
I: 2 Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has


llama_print_timings:        load time =     219.93 ms
llama_print_timings:      sample time =      31.70 ms /   256 runs   (    0.12 ms per token,  8075.46 tokens per second)
llama_print_timings: prompt eval time =    4311.70 ms /   149 tokens (   28.94 ms per token,    34.56 tokens per second)
llama_print_timings:        eval time =   36258.05 ms /   255 runs   (  142.19 ms per token,     7.03 tokens per second)
llama_print_timings:       total time =   41228.42 ms /   404 tokens


[array([[0.64754226]]), array([[0.7602322]]), array([[0.68960831]]), array([[0.75315796]]), array([[0.50718729]]), array([[0.72355241]]), array([[0.70556111]]), array([[0.73337518]])]
(0, array([[2.24754226]]))
(1, array([[2.4602322]]))
(2, array([[1.88960831]]))
(3, array([[2.25315796]]))
(4, array([[1.85718729]]))
(5, array([[2.67355241]]))
(6, array([[2.15556111]]))
(7, array([[1.93337518]]))
[(5, array([[2.67355241]])), (1, array([[2.4602322]])), (3, array([[2.25315796]])), (0, array([[2.24754226]])), (6, array([[2.15556111]])), (7, array([[1.93337518]])), (2, array([[1.88960831]])), (4, array([[1.85718729]]))]
{[6] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Marissa Roswell: "Her lover would die and never love another." "}
{[2] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Par

In [7]:
for obs in range(len(observations1)-1):
    print(cosine_similarity(observations1[obs].get_embed_vector(), observations1[obs+1].get_embed_vector().T))

print(cosine_similarity(observations1[obs].get_embed_vector(), get_llama_embedding("My teacher always to flunk me till junior high. thanks alot, next semester ill be 35.").T))

[[0.87151981]]
[[0.88036487]]
[[0.89643193]]
[[0.8845694]]
[[0.84738461]]
[[0.89728706]]
[[0.92602138]]
[[0.9373653]]



llama_print_timings:        load time =     438.75 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    1178.09 ms /    25 tokens (   47.12 ms per token,    21.22 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    1178.52 ms /    26 tokens


[[0.53871299]]


In [3]:

imp = (importance_f("{[3] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: \"I agree with Cassandra's noticing \"}", "{[4] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: \"of the author's word choice of barbarism.\"}"))

print(imp)

R

Llama.generate: prefix-match hit


ationale: These two messages are from the same person Paige Tyrone and related to the same question but they don't seem to be a continuation or adding more information to each other like in the previous example. Instead they seem to be two separate thoughts or replies to Cassandra Winfred's and Marissa Roswell's answers to the same question. Therefore, while they're related to each other in context they don't share direct sequential connection. Hence, the importance of one to the other is not very high.

I: 5
Rationale: While both the Observation and Query seem to be related to the same question, they aren't necessarily dependent on each other for understanding or importance. The Observation adds a commentary on the author's word choice while the Query seems to be more focused on seeking agreement or reaction to Cassandra Winfred's answer. They are related but not in a way that the importance of one to the other is very high. Hence, I assign a moderate importance value of 5 to this cas


llama_print_timings:        load time =     215.13 ms
llama_print_timings:      sample time =      27.92 ms /   227 runs   (    0.12 ms per token,  8129.21 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   32266.44 ms /   227 runs   (  142.14 ms per token,     7.04 tokens per second)
llama_print_timings:       total time =   32830.36 ms /   228 tokens


5.0
